# Finalise facebook comments topics
Finalises the facebook comments topics.
And persists it to s3.
See docs for more information.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import datetime

import pandas as pd
import tentaclio

from phoenix.common import artifacts, run_datetime
from phoenix.common import utils
from phoenix.tag import export
from phoenix.tag import finalise
from phoenix.tag import object_filters

In [ ]:
utils.setup_notebook_output()
utils.setup_notebook_logging()

In [ ]:
# Parameters
# See phoenix/common/run_datetime.py expected format of parameter
RUN_DATETIME = None

# See phoenix/common/artifacts/registry_environment.py expected format of parameter
ARTIFACTS_ENVIRONMENT_KEY = "local"

# Filters for batch
YEAR_FILTER = 2021
# Without zero padding
MONTH_FILTER = 8
OBJECT_TYPE = "facebook_comments"

# OUTPUT
FINAL_URL_FACEBOOK_COMMENTS_TOPICS = None

In [ ]:
if RUN_DATETIME:
    run_dt = run_datetime.from_file_safe_str(RUN_DATETIME)
else:
    run_dt = run_datetime.create_run_datetime_now()
    
url_config = {
    "YEAR_FILTER": YEAR_FILTER,
    "MONTH_FILTER": MONTH_FILTER,
    "OBJECT_TYPE": OBJECT_TYPE
}
art_url_reg = artifacts.registry.ArtifactURLRegistry(run_dt, ARTIFACTS_ENVIRONMENT_KEY, artifacts.registry_mappers.get_default_mappers())
TAGGING_RUNS_URL_TOPICS = art_url_reg.get_url("tagging_runs-topics", url_config)
TAGGING_RUNS_URL_FACEBOOK_COMMENTS_FINAL = art_url_reg.get_url("tagging_runs-facebook_comments_final", url_config)
TAGGING_RUNS_URL_FACEBOOK_COMMENTS_TOPICS_FINAL = art_url_reg.get_url("tagging_runs-facebook_comments_topics_final", url_config)
if not FINAL_URL_FACEBOOK_COMMENTS_TOPICS:
    FINAL_URL_FACEBOOK_COMMENTS_TOPICS = art_url_reg.get_url("final-facebook_comments_topics", url_config)

In [ ]:
# Display params.
print(
TAGGING_RUNS_URL_TOPICS,
TAGGING_RUNS_URL_FACEBOOK_COMMENTS_FINAL,
TAGGING_RUNS_URL_FACEBOOK_COMMENTS_TOPICS_FINAL,
FINAL_URL_FACEBOOK_COMMENTS_TOPICS,
run_dt.dt,
sep='\n',
)

In [ ]:
# %env DASK_CLUSTER_IP=

In [ ]:
utils.dask_global_init()

In [ ]:
facebook_comments_topics_df = artifacts.dataframes.get(TAGGING_RUNS_URL_FACEBOOK_COMMENTS_FINAL).dataframe

In [ ]:
facebook_comments_topics_df.head()

In [ ]:
topics = artifacts.dataframes.get(TAGGING_RUNS_URL_TOPICS).dataframe

In [ ]:
topics.head()

In [ ]:
facebook_comments_topics_final = finalise.join_topics_to_facebook_comments(topics, facebook_comments_topics_df)

In [ ]:
facebook_comments_topics_final.head()

In [ ]:
facebook_comments_topics_final.dtypes

In [ ]:
_ = artifacts.dataframes.persist(FINAL_URL_FACEBOOK_COMMENTS_TOPICS, facebook_comments_topics_final)

In [ ]:
_ = artifacts.dataframes.persist(TAGGING_RUNS_URL_FACEBOOK_COMMENTS_TOPICS_FINAL, facebook_comments_topics_final)

In [ ]:
artifacts.dataframes.read_schema(TAGGING_RUNS_URL_FACEBOOK_COMMENTS_TOPICS_FINAL)